# Cifrar <-> Decifrar Archivos de Texto-Binario

### Importamos libreria

In [1]:
import hashlib

### Creamos y leemos el archivo binario

In [2]:
file = open('testing.txt', 'bw')
file.write(b'algun texto')
file.close()
file = open('testing.txt', 'br')
content_global = file.read()
file.close()
print(content_global)

b'algun texto'


### Definimos el alfabeto a 26 carácteres y transformamos a unicode

In [3]:
alf = "abcdefghijklmnopqrstuvwxyz"
af = list(alf)
alfabeto = []
print(af)

for char in af:
    alfabeto.append(ord(char))

print(alfabeto)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]


### Función para cifrar el archivo

In [4]:
def cifrar(content, shift, alfabeto):
    data = list(content) #se crea una lista en unicode con el contenido en binario
    res = []

    print(f'Unicode inicial {data}')

    for char in data: # ciclo para aumentar el shift al unicode y cambiar el valor inicial
        if char in alfabeto:
            new_char = alfabeto[((alfabeto.index(char) + shift) % 26)]
            res.append(new_char)
        else:
            res.append(char)
    print(f'Unicode shifteado {res}')

    return bytes(res)

### Funcion para decifrar archivo

In [5]:
def decifrar(content, shift, alfabeto):
    data = list(content) #se crea una lista en unicode con el contenido en binario
    res = []

    print(f'Unicode inicial cifrado {data}')

    for char in data: # ciclo para aumentar el shift al unicode y cambiar el valor inicial
        if char in alfabeto:
            new_char = alfabeto[((alfabeto.index(char) - shift) % 26)]
            res.append(new_char)
        else:
            res.append(char)
    print(f'Unicode shifteado decifrado {res}')

    return bytes(res)


### Comprobar integridad de hash

In [6]:
def integrity(decoded_hash, og_hash):
    if decoded_hash == og_hash:
        return True
    else:
        return False
    

### Funciones para el archivo

In [7]:
def cifrarArchivo(path_in, path_out, shift):

    file = open(path_in, 'br')
    content = file.read()
    file.close()

    #hasheamos el contenido del txt
    hasher = hashlib.sha256()
    hasher.update(content) 
    binhash = hasher.digest() #bytes hash

    #unimos el hash con el contenido para cifrarlo todo
    cont = binhash + content
    print(f'Contenido inicial {cont}')

    #ciframos el contenido
    cifrado = cifrar(cont, shift, alfabeto) 
    file2 = open(path_out, 'bw')
    file2.write(cifrado) #Después escribimos el mismo contenido pero cifrado con el hash
    file.close()

def decifrarArchivo(path_in, path_out, shift):

    file = open(path_in, 'br')
    content = file.read()
    file.close()

    #Decifrar contenido y quedarnos con el hash
    cifrado = decifrar(content, shift, alfabeto)
    hash_unique = cifrado[:32]
    print(hash_unique)

    #quedarnos con el contenido binario
    content_unique = cifrado[32:]
    print(content_unique)

    #hasheamos el contenido del txt
    hasher = hashlib.sha256()
    hasher.update(content_unique)
    binhash = hasher.digest() #bytes hash

    #Se verifica si el contenido sufrió algún cambio
    if integrity(hash_unique, binhash) == True:
       file2 = open(path_out, 'bw')
       file2.write(content_unique)
       file.close()
       print("La integridad fue verificada exitosamente")
    else:
        raise RuntimeError(f'Los datos del archivo {path_in} han sido corrompidos')


    

In [13]:
# path_in = "./testing.txt"
# path_out = "./testingcode.txt"

# cifrarArchivo(path_in, path_out, 20)

path_in = "./testingcode.txt"
path_out = "./testingdecode.txt"

decifrarArchivo(path_in, path_out, 20)

Unicode inicial cifrado [79, 239, 191, 189, 239, 191, 189, 239, 191, 189, 31, 239, 191, 189, 13, 10, 127, 124, 73, 239, 191, 189, 5, 239, 191, 189, 239, 191, 189, 222, 149, 33, 91, 239, 191, 189, 85, 24, 83, 239, 191, 189, 125, 108, 239, 191, 189, 239, 191, 189, 239, 191, 189, 57, 96, 239, 191, 189, 239, 191, 189, 117, 102, 97, 111, 104, 32, 110, 121, 114, 110, 105, 120, 115]
Unicode shifteado decifrado [79, 239, 191, 189, 239, 191, 189, 239, 191, 189, 31, 239, 191, 189, 13, 10, 127, 124, 73, 239, 191, 189, 5, 239, 191, 189, 239, 191, 189, 222, 149, 33, 91, 239, 191, 189, 85, 24, 83, 239, 191, 189, 125, 114, 239, 191, 189, 239, 191, 189, 239, 191, 189, 57, 96, 239, 191, 189, 239, 191, 189, 97, 108, 103, 117, 110, 32, 116, 101, 120, 116, 111, 100, 121]
b'O\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\x1f\xef\xbf\xbd\r\n\x7f|I\xef\xbf\xbd\x05\xef\xbf\xbd\xef\xbf\xbd\xde\x95!'
b'[\xef\xbf\xbdU\x18S\xef\xbf\xbd}r\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd9`\xef\xbf\xbd\xef\xbf\xbdalgun textody'


RuntimeError: Los datos del archivo ./testingcode.txt han sido corrompidos